# Load Clean CSV Files Onto a Postgres Database (Neon)

In [7]:
import os
from dotenv import load_dotenv

# Initialize .env
load_dotenv()

True

In [8]:
# Load DB URL from .env file
DATABASE_URL = os.getenv("DATABASE_URL")

# Create the SQLAlchemy Engine

In [10]:
from sqlalchemy import create_engine

# Establish connection
engine = create_engine(DATABASE_URL)

# Test
engine.connect()